# Error bars

Given the formula that links the differential cross section to the event rate measured by our experiment:

$$
\frac{dN}{dt\,d \Omega_{spect}} = \epsilon_{spect}\left(\theta \right) \cdot \frac{N_{gate}}{T} \cdot n_{c} \cdot \wp(\theta; \lambda', \lambda '') \cdot \frac{d \sigma}{d \Omega}
$$
$$
\frac{dN}{dt\,d \Omega_{spect}} \approx \frac{N}{\Delta T\,\Omega_{spect}} := m \hspace{1cm} \frac{N}{\Delta T} := R
$$

with our equipment we were able to measure just the rate of events for a certain scattering angle, not the $\Omega_{spect}$, which is computed by a MC simulation. 

Once settled this key concept it is crucial to correctly evaluate the uncertainities linked to these measurements. 

## Statistical and systematical uncertainties + work flow
First thing first it is fundamental to distinguish between the various sources of errors one can come across during a measurement in a nuclear physics' experiment.
In particular we need to separate the contributions given by $\textit{statistical}$ and $\textit{systematical}$ uncertainties:

- Statistical uncertainities could come from the counting of photons or the errors associated to the fit parameters, a square sum will be used to take account of them.
- Systematical uncertainties arise from various effects, such as the shift of the Compton peak, the errors associated with the geometrical measurements of the apparatus and so on, we'll take account of them with a square sum as well.

To make sure randomness and bias are not mixed together these two kinds of uncertainties will be combined as:

$$
\delta_{tot} = \delta_{statistic} + \delta_{systematic}
$$

so that one can split the two contributes in:

$$ \left(\frac{\delta Q_{stat}}{Q} \right)^2 = \sum_{X \, \in \, (d \Omega_{spect}, \epsilon_{spect}, n_{gate}, n_c, \wp)}  \left(\frac{\delta X_{stat}}{X} \right)^2 $$
$$ \left(\frac{\delta Q_{sist}}{Q} \right)^2 = \sum_{X \, \in \, (d \Omega_{spect}, \epsilon_{spect}, n_{gate}, n_c, \wp)}  \left(\frac{\delta X_{sist}}{X} \right)^2 $$


By looking at the expression above one could come up with an expression for the uncertainty that looks like:

$$
\delta m= \underbrace{\left(\frac{\partial m}{\partial R} \right)\delta R \oplus \frac{1}{\sqrt{N}} \oplus \left(\frac{\partial m}{\partial N} \right) \delta N}_{\delta_{statistic}} + \underbrace{\left(\frac{\partial m}{\partial \Omega_{spect}} \right) \delta \Omega_{spect} \oplus m\left[\left(\frac{\delta \epsilon_{spect}}{\epsilon_{spect}} \right) \oplus \left(\frac{\delta N_{gate}}{N_{gate}} \right) \oplus \left(\frac{\delta n_c}{n_c} \right) \oplus \left(\frac{\delta \wp}{\wp} \right)\right]}_{\delta_{systematic}}
$$

assuming every quantity considered in this expression is independent from one another and has a relatively small error associated to it.


## Systematic uncertainties
### $n_c$ uncertainty

Given:

$$
n_c = \rho \frac{N_a \cdot Z}{MM}
$$

where: 
- $\rho$ is the density of the scattering target
- $N_a$ is the Avogadro number
- $\text{MM}$ is the molar mass of the scattering target

One can clearly see that all these quantities are known with extreme accuracy, we won't take account of their uncertainties.

### $\epsilon_{spect}$ uncertainty

In this case the value of $\epsilon_{spect}$ is obtained by an interpolation, the error associated to it will then be consider as a statistical uncertainty. Given the formula used to fit the efficiency curve: 

$$
\epsilon_{spect} = A \cdot E^{-B} \cdot \exp(-C \cdot E) + D
$$

where $A,B,C,D$ are all model parameters and $E$ is the energy of the incoming gamma ray. Given that, one can compute the error associated to this quantity as:

$$
\delta \epsilon = \left( \frac{\partial \epsilon}{\partial A} \cdot \delta A \right) \oplus \left( \frac{\partial \epsilon}{\partial B} \cdot \delta B \right) \oplus \left( \frac{\partial \epsilon}{\partial C} \cdot \delta C \right) \oplus \left( \frac{\partial \epsilon}{\partial D} \cdot \delta D \right)
$$

$$
\delta \epsilon = \sqrt{\left[E^{-B} \cdot e^{-CE} \cdot \delta A \right]^2 + \left[A \cdot E^{-B} \cdot e^{-CE} \cdot \ln(B) \cdot \delta B \right]^2 + \left[A \cdot E^{-B} \cdot e^{-CE} \cdot E \cdot \delta C \right]^2 + \left[ \delta D \right]^2}
$$



In [17]:
import numpy as np

# Parametri del fit e le relative incertezze
A = 1.5604
delta_A = 1.2255

B = -0.0995
delta_B = 0.04328

C = 3.53466
delta_C = 1.00558

D = 0.10210
delta_D = 0.00978

def epsilon_spettrometer(theta): 

    A = 1.5604
    B = -0.0995
    C = 3.53466
    D = 0.10210
    
    E  = 511/(2 - np.cos(theta)) * 1e-3 #da MeV a keV

    return A * pow(E, -B) * np.exp(-C * E) + D


def delta_eff(E):
    term1 = E**(-2 * B) * np.exp(-2 * C * E) * delta_A**2
    term2 = A**2 * E**(-2 * B) * np.exp(-2 * C * E) * (np.log(E))**2 * delta_B**2
    term3 = A**2 * E**(-2 * B) * np.exp(-2 * C * E) * (E**2) * delta_C**2 
    term4 = delta_D**2

    delta_epsilon = np.sqrt(term1 + term2 + term3 + term4)
    return delta_epsilon

E_input = 511 
result = delta_eff(E_input)
print(f"\nIncertezza sull'efficienza (δϵ) a E = {E_input:.2f} è {result:.6f}")



Incertezza sull'efficienza (δϵ) a E = 511.00 è 0.009780


### $N_{gate}$ uncertainty

Given: 

$$
N_{gate} = 2 \cdot S \left(t \right) \cdot \text{BR} \cdot \frac{\Delta\Omega}{4\pi} \cdot \epsilon_{gate}(511)
$$

where: 
- $S(t)$ is the activity of the source in Bq
- $\text{BR}$ is the branching ratio of the 511 keV photon
- $\frac{\Delta\Omega}{4\pi}$ is the solid angle
- $\epsilon_{gate}(511)$ is the efficiency of the gate detector for the 511 keV photon

In this case one can assume that the branching ratio (BR) is well known, the uncertainties linked to $S \left(t \right)$ and $\epsilon_{gate}$ are statistical errors and the one associated to $\Delta \Omega$ is systematical.

#### $\delta S(t)$ computation
For this uncertainty, an error of 0.5 cm in the distance from the detector and 3 mm in the displacement from the detector axis was considered. The acquisition times were assumed to be error-free, as they were set within the Maestro application. For the number of counts, the uncertainty associated with Poisson statistics was taken into account.
The associated error is then:

$$
\delta S \left( t \right) = 11647 \ Bq.
$$

#### $\delta \epsilon_{gate}$ computation

For this computation please refer to the previous section.
We impose a 2.5% error on each point and from the linear fit we gain:

$$
\delta \epsilon_{gate} (511) = 0.01672   
$$

#### $\delta (\Delta \Omega / 4\pi)$ computation

For the uncertainty linked to the measurement of the solid angle covered by the spectrometer one can start from the formula:

$$
\frac{\Delta\Omega}{4\pi} =  \frac{1 - \cos (\beta)}{2}
$$

where $\beta$ is computed as: 

$$
\beta = \arctan \left( \frac{r_{gate}}{d_{source-gate}} \right).
$$

Both of these two measurements are known with their uncertainty, $r_{gate} = (1,27 \pm 0,01) \, cm$ and ${d_{source-gate}} = d =(18.54 \pm 0.5) \, cm$, knowing that one can propagate the errors using the error propagation expression.

$$
\left(\delta (\Delta \Omega / 4\pi) \right) = \left( \frac{\partial (\Delta \Omega / 4\pi)}{\partial r} \delta r\right) \oplus \left( \frac{\partial (\Delta \Omega / 4\pi)}{\partial d} \delta d\right)
$$

and that:

$$
\frac{\partial (\Delta \Omega / 4\pi)}{\partial r} = \frac{r}{2 d^2 \cdot \left( \frac{r^2}{d^2} + 1 \right)^{3/2}}
$$

$$
\frac{\partial (\Delta \Omega / 4\pi)}{\partial d} = - \frac{r^2}{2 d^3 \cdot \left( \frac{r^2}{d^2} + 1 \right)^{3/2}}
$$

one can find the uncertainty asssociated to the solid angle using the formula below:

$$
\delta \left(\Delta \Omega / 4\pi \right) = \sqrt{\left(\frac{r}{2 d^2 \cdot \left( \frac{r^2}{d^2} + 1 \right)^{3/2}} \cdot \delta r \right)^2 + \left(\frac{r^2}{2 d^3 \cdot\left( \frac{r^2}{d^2} + 1 \right)^{3/2}} \cdot \delta d \right)^2}
$$


In [18]:
import math

# Valori noti (convertiti in metri se vuoi usare il SI, qui rimangono in cm)
r = 1.27       # cm
delta_r = 0.01 # cm
d = 18.54       # cm
delta_d = 0.5  # cm

term1  = r * delta_r / (2 * d**2 * (r**2/d**2 + 1)**(3/2))
term2 = r**2 * delta_d / (2 * d**3 * (r**2/d**2 + 1)**(3/2))

delta_Omega = np.sqrt(term1**2 + term2**2)

Omega = 2 * np.pi * (1-np.cos(np.arctan(r/d)))

print(f"Incertezza sulla frazione di angolo solido ΔΩ/4π:",Omega ,"+/-", delta_Omega)

BR = 0.903
epsilon = 0.147
delta_epsilon = 0.01672
S = 188900 
delta_S = 11647

Ngate = 2 * BR * epsilon * S * Omega
delta_Ngate = np.sqrt((delta_epsilon / epsilon) ** 2 + (delta_S / S) ** 2 + (delta_Omega / Omega) ** 2)
print(f"\nNumero di eventi nella regione di gate (Ngate): {Ngate:.2f} ± {delta_Ngate:.2f}")

Incertezza sulla frazione di angolo solido ΔΩ/4π: 0.01468969384187046 +/- 6.545346664430829e-05

Numero di eventi nella regione di gate (Ngate): 736.68 ± 0.13


### $d \Omega_{spect}$ uncertainty

In questa sezione faccio riferimento al disegno che ha fatto Andre per il calcolo dell'angolo solido.

Since we are computing a solid angle, the formula used to compute it is the same as always:

$$
d \Omega = 2 \pi \cdot \left( 1 - \cos \beta \right)
$$

$$
\delta \Omega_{spect} = \text{IQR}_{\Omega}
$$


In [19]:
def parse_measurement_file(filepath):
    from collections import defaultdict

    # Initialize dictionary containers
    solid_angles_reflection = {}
    solid_angles_transmission = {}
    a_param_reflection = {}
    a_param_transmission = {}
    b_param_reflection = {}
    b_param_transmission = {}

    # Mapping section and mode to corresponding dict
    section_map = {
        ('Solid Angles', 'Reflection'): solid_angles_reflection,
        ('Solid Angles', 'Trasmission'): solid_angles_transmission,
        ('A param', 'Reflection'): a_param_reflection,
        ('A param', 'Trasmission'): a_param_transmission,
        ('B param', 'Reflection'): b_param_reflection,
        ('B param', 'Trasmission'): b_param_transmission,
    }

    current_section = None
    current_mode = None
    collecting_data = False

    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()

            if not line or set(line) == {'*'}:
                collecting_data = False
                continue

            # Section headers
            if line in {'Solid Angles', 'A param', 'B param'}:
                current_section = line
                current_mode = None  # reset until mode is found
                continue

            # Mode headers
            if line in {'Reflection', 'Trasmission'}:
                current_mode = line
                continue

            # Header line before data
            if line.startswith('Angle Mean Median'):
                collecting_data = True
                continue

            # Data lines
            if collecting_data and current_section and current_mode:
                parts = line.split()
                if len(parts) == 5:
                    angle = int(parts[0])
                    mean = float(parts[1])
                    median = float(parts[2])
                    std = float(parts[3])
                    iqr = float(parts[4])
                    section_map[(current_section, current_mode)][angle] = [mean, median, std, iqr]

    return (
        solid_angles_reflection,
        solid_angles_transmission,
        a_param_reflection,
        a_param_transmission,
        b_param_reflection,
        b_param_transmission,
    )

SA_riflection, SA_trasmission, A_riflection, A_trasmission, B_riflection, B_trasmission = parse_measurement_file("solid_angle.txt")

delta =  0.025
C = 5.08
delta_C = 0.01

dictionary = {"Mean": 0, "Median": 1, "STD": 2, "IQR": 3}

trasmission_angles = [35, 40, 50, 60]
riflection_angles = [40, 50, 60, 70, 80, 90, 100, 110]

Omega_riflection = [SA_riflection[angle][dictionary['Median']] for angle in riflection_angles]
Omega_riflection_err = [SA_riflection[angle][dictionary['IQR']] for angle in riflection_angles]

Omega_trasmission = [SA_trasmission[angle][dictionary['Median']] for angle in trasmission_angles]
Omega_trasmission_err = [SA_trasmission[angle][dictionary['IQR']] for angle in trasmission_angles]

delta_Omega_riflection = Omega_riflection_err
delta_Omega_trasmission = Omega_trasmission_err

print("\nIncertezze sulla frazione di angolo solido per la riflessione:\n", delta_Omega_riflection)
print("\nIncertezze sulla frazione di angolo solido per la trasmissione:\n", delta_Omega_trasmission)


Incertezze sulla frazione di angolo solido per la riflessione:
 [0.001476859791490797, 0.001826765948870511, 0.00174062856617508, 0.0011169944695525735, 0.0004845506083612887, 0.0005718801675267428, 0.0010008257127981542, 0.0011133157697127312]

Incertezze sulla frazione di angolo solido per la trasmissione:
 [0.0008435438465544606, 0.0006416314210639269, 0.0008892136137358744, 0.0008897971792260553]


We decide to use just the statistical error on the $\Omega_{spect}$.


### Result systematic uncertainities

$$
\delta_{systematic} = \left(\frac{\partial m}{\partial \Omega_{spect}} \right) \delta \Omega_{spect} \oplus m\left[\left(\frac{\delta \epsilon_{spect}}{\epsilon_{spect}} \right) \oplus \left(\frac{\delta n_{gate}}{n_{gate}} \right) \oplus \left(\frac{\delta n_c}{n_c} \right) \oplus \left(\frac{\delta \wp}{\wp} \right)\right]
$$

where:
$$
\frac{\partial m}{\partial \Omega_{spect}} = \frac{\partial }{\partial \Omega_{spect}}\bigg(\frac{R}{\Omega_{spect}}\bigg) = -\frac{R}{\Omega_{spect}^2}
$$

In [20]:
def leggi_dati(file_path):
    # Inizializza liste per ogni colonna
    angle, rate, err_rate = [], [], []
    count, err_count = [], []
    channel, err_channel = [], []
    sigma, err_sigma = [], []

    with open(file_path, 'r') as file:
        lines = file.readlines()

        # Salta l'intestazione
        for line in lines[1:]:
            valori = line.strip().split()
            if len(valori) != 9:
                continue  # Salta righe non valide

            angle.append(float(valori[0]))
            rate.append(float(valori[1]))
            err_rate.append(float(valori[2]))
            count.append(float(valori[3]))
            err_count.append(float(valori[4]))
            channel.append(float(valori[5]))
            err_channel.append(float(valori[6]))
            sigma.append(float(valori[7]))
            err_sigma.append(float(valori[8]))

    return angle, rate, err_rate, count, err_count, channel, err_channel, sigma, err_sigma

file_path = "../Codes/data_analysis/parameters_pol4_riflection.txt"

angle_riflection, rate_riflection, err_rate_riflection, count_riflection, err_count_riflection, channel_riflection, err_channel_riflection, sigma_riflection, err_sigma_riflection = leggi_dati(file_path)

file_path = "../Codes/data_analysis/parameters_pol4_trasmission.txt"

angle_trasmission, rate_trasmission, err_rate_trasmission, count_trasmission, err_count_trasmission, channel_trasmission, err_channel_trasmission, sigma_trasmission, err_sigma_trasmission = leggi_dati(file_path)

In [21]:
epsilon_riflection = [epsilon_spettrometer(np.radians(angle)) for angle in angle_riflection]
epsilon_trasmission = [epsilon_spettrometer(np.radians(angle)) for angle in angle_trasmission]

m_riflection = [rate / Omega for rate, Omega in zip(rate_riflection, Omega_riflection)]
m_trasmission = [rate / Omega for rate, Omega in zip(rate_trasmission, Omega_trasmission)]

der_m_riflection = [rate / (Omega ** 2) for rate, Omega in zip(rate_riflection, Omega_riflection)]
print("Derivata di m per la riflessione:", der_m_riflection)
der_m_trasmission = [rate / (Omega ** 2) for rate, Omega in zip(rate_trasmission, Omega_trasmission)]
print("Derivata di m per la trasmissione:", der_m_trasmission)

delta_systematic_riflection = []
for rate, m, Omega, epsilon, der_m, err_Omega in zip(rate_riflection, m_riflection, Omega_riflection, epsilon_riflection, der_m_riflection, delta_Omega_riflection):
    term1 = der_m * err_Omega
    term2 = m * delta_eff(511) / epsilon
    term3 = m * delta_Ngate / Ngate
    print(term1, term2, term3)
    delta_systematic_riflection.append(np.sqrt(term1**2 + term2**2 + term3**2))

print("\nIncertezze sistematiche per la riflessione:\n", delta_systematic_riflection)
print("\n")

delta_systematic_trasmission = []
for rate, m, Omega, epsilon, der_m, err_Omega in zip(rate_trasmission, m_trasmission, Omega_trasmission, epsilon_trasmission, der_m_trasmission, delta_Omega_trasmission):
    term1 = der_m * err_Omega
    term2 = m * delta_eff(511) / epsilon
    term3 = m * delta_Ngate / Ngate
    print(term1, term2, term3)
    delta_systematic_trasmission.append(np.sqrt(term1**2 + term2**2 + term3**2))

print("\nIncertezze sistematiche per la trasmissione:\n", delta_systematic_trasmission)

Derivata di m per la riflessione: [23.273355737923932, 65.22119705916829, 49.49424368953349, 35.8321848187814, 29.01185309113164, 21.01363565644385, 18.82254814524044, 20.57463541443413]
Derivata di m per la trasmissione: [32.140493172920785, 28.018228301372183, 20.346669316468887, 14.945278035304717]
0.03437148330240148 0.01438393409746974 0.000111858113034166
0.11914386193226215 0.03458081275452337 0.00029592233816506946
0.08615109442723268 0.02507451998410379 0.00023544513620403281
0.04002435227456451 0.01619859877273201 0.0001656202773063949
0.01405771106499617 0.012292526041885745 0.00013548690228320517
0.012017281479553044 0.008384095707630141 9.855986467197778e-05
0.018838090164137837 0.007015373102270587 8.705958715791526e-05
0.02290606606297955 0.007459319751280616 9.680564514587686e-05

Incertezze sistematiche per la riflessione:
 [np.float64(0.0372600179383353), np.float64(0.12406119464573753), np.float64(0.08972623951696261), np.float64(0.04317836040512246), np.float64(0.01

## Statistic uncertainities
$$
\delta_{statistic} = \left(\frac{\partial m}{\partial R} \right)\delta R \oplus \frac{1}{\sqrt{N}} \oplus \left(\frac{\partial m}{\partial N} \right) \delta N
$$
where: 
$$
\frac{\partial m}{\partial R} = \frac{1}{\Omega_{spect}} \hspace{1cm} \frac{\partial m}{\partial N}= \frac{1}{\Delta T \,\Omega_{spect}}
$$

In [22]:
T_riflection = [43000 * 13, 43000 * 6 + 40184, 43000 * 6 + 31193 + 10254, 16 * 43000, 23 * 43000, 22 * 43000 + 41203, 43000 * 17, 43000 * 14 + 29739]
T_trasmission = [43000 * 14, 43000 * 9, 43200 * 17 + 27346, 43000 * 13]

delta_statistic_riflection = []
for m, N, R, Omega, T, err_R, err_N in zip(m_riflection, count_riflection, rate_riflection, Omega_riflection, T_riflection, err_rate_riflection, err_count_riflection):
    term1 = (1 / Omega) * err_R
    term2 = (1 / np.sqrt(N))
    term3 = (1 / (Omega * T)) * err_N
    print(term1, term2, term3)
    delta_statistic_riflection.append(np.sqrt(term1**2 + term2**2 + term3**2))

print("\nIncertezze statistiche per la riflessione:\n", delta_statistic_riflection)
print("\n")

delta_statistic_trasmission = []
for m, N, R, Omega, T, err_R, err_N in zip(m_trasmission, count_trasmission, rate_trasmission, Omega_trasmission, T_trasmission, err_rate_trasmission, err_count_trasmission):
    term1 = (1 / Omega) * err_R
    term2 = (1 / np.sqrt(N))
    term3 = (1 / (Omega * T)) * err_N
    print(term1, term2, term3)
    delta_statistic_trasmission.append(np.sqrt(term1**2 + term2**2 + term3**2))

print("\nIncertezze statistiche per la trasmissione:\n", delta_statistic_trasmission)

0.004753061904819483 0.007621118909339405 0.008581281435424743
0.014717471777241128 0.00878203809420853 0.014794115426325995
0.012929198783980965 0.009595974526959023 0.01285438552007836
0.007223571607395093 0.0076576098793179935 0.00721694042170899
0.005267988173311732 0.00702489043056506 0.005417893369056699
0.004495944535324643 0.008225703812845509 0.004614947951396323
0.004939053891965807 0.010240667483882205 0.005072626116551225
0.005602235006929432 0.010358273741133068 0.005705053069581903

Incertezze statistiche per la riflessione:
 [np.float64(0.01242221566288809), np.float64(0.02264053930120649), np.float64(0.020602964238588133), np.float64(0.012763353982456578), np.float64(0.010317671903780686), np.float64(0.010448610676806452), np.float64(0.012449781507380056), np.float64(0.013085354504019308)]


0.007119622597456068 0.00851911148801243 0.007307603540315985
0.00863238456075467 0.01139776003596233 0.008505337894373492
0.005301313668373675 0.009616851795116267 0.00516985809584

## Final results

Knowing all the information we need, the uncertainty on the rate measurement can be expressed as:

$$
\delta m = \delta_{statistic} + \delta_{systematic}
$$

In [23]:
delta_total_riflection = []
for systematic, statistic in zip(delta_systematic_riflection, delta_statistic_riflection):
    print(systematic, statistic)
    delta_total_riflection.append(np.sqrt(systematic + statistic))

print("\nIncertezze totali per la riflessione:\n", delta_total_riflection)
print("\n")

delta_total_trasmission = []
for systematic, statistic in zip(delta_systematic_trasmission, delta_statistic_trasmission):
    print(systematic, statistic)
    delta_total_trasmission.append(np.sqrt(systematic + statistic))

print("\nIncertezze totali per la trasmissione:\n", delta_total_trasmission)

0.0372600179383353 0.01242221566288809
0.12406119464573753 0.02264053930120649
0.08972623951696261 0.020602964238588133
0.04317836040512246 0.012763353982456578
0.01867468322564121 0.010317671903780686
0.014653253189665254 0.010448610676806452
0.02010215610744192 0.012449781507380056
0.024090219695269713 0.013085354504019308

Incertezze totali per la riflessione:
 [np.float64(0.22289511793940978), np.float64(0.38301662359086197), np.float64(0.33215840160313687), np.float64(0.23652000842968665), np.float64(0.17027141606688392), np.float64(0.15843567737877634), np.float64(0.1804215552943217), np.float64(0.1928096838835877)]


0.03387395724070609 0.013291552053008375
0.02465284392678021 0.01663633883219506
0.02119194250158541 0.01213734723038322
0.015251071201359927 0.014907372935202973

Incertezze totali per la trasmissione:
 [np.float64(0.2171762171456959), np.float64(0.20319739850444754), np.float64(0.18256311164079295), np.float64(0.17366186724944224)]


# Angle uncertainties

On the x-axis of our Klein-Nishina plot we put the scattering angles, linked to these quantities there are a few uncertainty factors such as:
- The shift in the position of the Compton peak due to temperature instability and other environmental factors.
- The error associated to the measurement of the scattering angle. 
- The uncertainty linked to the spectrometer resolution, which, of course, is not ideal.
- The statistic error derived from the gaussian fit of the Compton peak.

While these first three factors have a systematic nature, the last one can be considered as strictly statistical. Knowing this it is crucial to transform every energy-related quantity into an angle-related quantity, one can do that simply by using the inverted Compton formula shown below:

$$
\theta (E') = \arccos \left[2- \frac{511}{E'} \right]
$$

that gives rise to the associated uncertainty formula, which is:

$$
\delta \theta = \left |\frac{511}{E'^2 \cdot \sqrt{1 - \left(2 - \frac{511}{E'} \right)^2}} \right | \cdot \delta E' 
$$







## Peak shift uncertainty

Knowing that the Compton peak undergoes a mean fluctuation called $\Delta E_{peak} = 8 \, ch = 24 \, keV$, we used $\delta E_{peak} = \frac{\Delta E_{peak}}{2} = 4 \, ch = 12 \,keV$ to quantify our peak energy uncertainty. This can be expressed as:

$$
\delta \theta_{peak} = \left |\frac{511}{E'^2 \cdot \sqrt{1 - \left(2 - \frac{511}{E'} \right)^2}} \right | \cdot \delta E_{peak}  .
$$

In [24]:
energies_riflection = 511/(2 - np.cos(np.radians(riflection_angles)))
energies_trasmission = 511/(2 - np.cos(np.radians(trasmission_angles)))

dE = 12 #keV
dThetaPeak =np.array([])

for i in range(len(energies_riflection)):
    dTheta = 511/(energies_riflection[i]**2 * np.sqrt(1 - (2 - 511/energies_riflection[i])**2)) * dE
    dThetaPeak = np.append(dThetaPeak, dTheta)
print(dThetaPeak)

dThetaPeak_trasm = np.array([])
for i in range(len(energies_trasmission)):
    dTheta = 511/(energies_trasmission[i]**2 * np.sqrt(1 - (2 - 511/energies_trasmission[i])**2)) * dE
    dThetaPeak_trasm = np.append(dThetaPeak_trasm, dTheta)
print(dThetaPeak_trasm)

[0.0556278  0.05646795 0.06101157 0.06869625 0.07953857 0.09393346
 0.11266458 0.13707422]
[0.0570896  0.0556278  0.05646795 0.06101157]


## Scattering angle uncertainty

This is chosen besed on the instrumental error used to measure $\theta_{scattering}$, and it is called $\delta \theta_{scattering}$.

Assuming $a = \arctan\bigg(\frac{r_{spectrometer}}{d_{target-spectrometer}}\bigg)$ one can compute the error assocaited to the placement of the detector at a given scattering angle as:

$$
\delta \theta_{scattering} = \frac{a}{\sqrt{12}}
$$


In [25]:
dThetaScattering = (np.arctan(2.54 / 27.54)) / np.sqrt(12)
print(dThetaScattering)

0.026549250386340584


## Resolution uncertainty

The error due to the resolution of a detector can be expressed by means of the percentile resolution of our spectrometer, that can be computed as:

$$
\delta_{resolution} = \frac{2.35 \cdot \sigma_{fit}}{channel} \cdot 100
$$

Just like the previous case, this quantity needs to be expressed in terms of the scattering angle, this can be computed using:

$$
\delta \theta_{resolution} = \left| \frac{1}{sin^2 \theta \cdot (E')^2}\right| \cdot \delta_{resolution} 
$$

In [26]:
sigma_resolution_riflection = np.array(sigma_riflection) * 2.35 / channel_riflection * 100
dThetaRis = 1/(np.sin(np.radians(riflection_angles))**2 * (energies_riflection**2)) * sigma_resolution_riflection
print(dThetaRis)

sigma_resolution_trasmission = np.array(sigma_trasmission) * 2.35 / channel_trasmission * 100
dThetaRis_trasm = 1/(np.sin(np.radians(trasmission_angles))**2 * (energies_trasmission**2)) * sigma_resolution_trasmission
print(dThetaRis_trasm)

[0.000153   0.00015296 0.00015002 0.00014502 0.00015681 0.00017769
 0.0002197  0.00023404]
[0.00016003 0.00014937 0.00013909 0.00014424]


## Statistical uncertainty

The error derived solely from the fit of the Compton peak can be taken in account with the formula:

$$
\delta_{fit} = \frac{\sigma_{fit}}{\sqrt{N}}
$$

Where N is the number of elements in my Compton peak, here as well one needs to express this error in terms of the scattering angle, the formula used is the same:

$$
\delta \theta_{fit} = \left| \frac{1}{sin^2 \theta \cdot (E')^2}\right| \cdot \delta_{fit}.
$$

In [27]:
sigma_E_fit_riflection = np.array(sigma_riflection) / np.sqrt(count_riflection)
dThetaFit = 1/(np.sin(np.radians(riflection_angles))**2 * (energies_riflection**2)) * sigma_E_fit_riflection
print(dThetaFit)

sigma_E_fit_trasmission = np.array(sigma_trasmission) / np.sqrt(count_trasmission)
dThetaFit_trasm = 1/(np.sin(np.radians(trasmission_angles))**2 * (energies_trasmission**2)) * sigma_E_fit_trasmission
print(dThetaFit_trasm)

[5.97912529e-06 6.32787590e-06 6.17826692e-06 4.31086365e-06
 3.88239037e-06 4.77310765e-06 6.68889164e-06 6.77622623e-06]
[7.42561022e-06 9.11902085e-06 6.44154328e-06 8.15204291e-06]


## Final results

One can express the error bars on the x-axis as a sum of statistical and systematical uncertainties following the expression:

$$
\delta \theta = \delta \theta_{stat} \pm \delta \theta_{syst} = (\delta \theta_{fit}) \pm \left(\delta \theta_{peak} \oplus \delta \theta_{scattering} \oplus \delta \theta_{resolution} \right)
$$

In [28]:
dThetaTOT = dThetaFit + np.sqrt(dThetaPeak**2 + dThetaRis**2 + dThetaScattering**2)
print (dThetaTOT)

dThetaTOT_trasm = dThetaFit_trasm + np.sqrt(dThetaPeak_trasm**2 + dThetaRis_trasm**2 + dThetaScattering**2)
print(dThetaTOT_trasm)

[0.06164475 0.06240437 0.06654412 0.07365252 0.08385656 0.09761824
 0.11575736 0.13962862]
[0.06296861 0.06164788 0.06240445 0.06654608]


In [29]:
np.savez('errori_arrays_riflex.npz', y_err=delta_total_riflection, x_err=dThetaTOT)
np.savez('errori_arrays_trasm.npz', y_err=delta_total_trasmission, x_err=dThetaTOT_trasm)

print("File 'errori_arrays.npz' salvato con successo.")

File 'errori_arrays.npz' salvato con successo.


# Energies uncertainity

$$
\delta E = \delta E_{shift} \pm (\delta E_{resolution} \oplus \delta E_{fit})
$$

In [30]:
delta_energies_riflection = dE + np.sqrt(sigma_E_fit_riflection ** 2 + sigma_resolution_riflection ** 2)
delta_energies_trasmission = dE + np.sqrt(sigma_E_fit_trasmission ** 2 + sigma_resolution_trasmission ** 2)

# Angles uncertainity (Energy plot)

In [31]:
delta_angle = dThetaScattering

In [32]:
np.savez('errori_arrays_energies_riflex.npz', y_err=delta_energies_riflection, x_err=delta_angle)
np.savez('errori_arrays_energies_trasm.npz', y_err=delta_energies_trasmission, x_err=delta_angle)

print("File 'errori_arrays.npz' salvato con successo.")

File 'errori_arrays.npz' salvato con successo.
